---
title: Parameter calculation of available data
subtitle: Data is obtained from [FRBSTATS](https://www.herta-experiment.org/frbstats/)
author: Murthadza Aznam
date: '2022-11-24'
---

:::{.callout-note}

 📌 Goal: This notebook tries to calculate some missing parameters from FRBSTATS using methods described in [Jia-Wei Luo (2022)](https://doi.org/10.1093/mnras/stac3206)

:::

In [28]:
from pathlib import Path

import pandas as pd
import numpy as np
import scipy

## 0. Getting The Data

### 0.1 Source

This notebook uses data from [FRBSTATS](https://www.herta-experiment.org/frbstats/)

In [2]:
datapath = Path('..', 'data')
external_datapath = Path(datapath, 'raw', 'external')

In [46]:
catalog: pd.DataFrame = pd.read_csv(Path(external_datapath, "FRBSTATS2022-11-23_population.csv")).replace("-", None)
# Labeling repeaters
rptrs: pd.DataFrame = pd.read_csv(Path(external_datapath, "FRBSTATS2022-11-23_repeaters.csv"))
catalog["label"] = [
    "repeater"
    if name in [*rptrs["name"].to_list(), *rptrs["samples"].to_list()]
    else "non-repeater"
    for name in catalog["frb"]
]
catalog["repeater"] = [
    False if name == "non-repeater" else True for name in catalog["label"]
]
catalog = catalog.dropna(axis=0, subset=['flux', 'fluence', 'redshift', 'frequency']).astype({'flux': float, 'fluence': float, 'width': float, 'redshift': float})
catalog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 0 to 813
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   frb                636 non-null    object 
 1   utc                636 non-null    object 
 2   mjd                636 non-null    float64
 3   telescope          636 non-null    object 
 4   ra                 636 non-null    object 
 5   dec                636 non-null    object 
 6   l                  636 non-null    float64
 7   b                  636 non-null    float64
 8   frequency          636 non-null    float64
 9   dm                 636 non-null    float64
 10  flux               636 non-null    float64
 11  width              634 non-null    float64
 12  fluence            636 non-null    float64
 13  snr                634 non-null    object 
 14  reference          636 non-null    object 
 15  redshift           636 non-null    float64
 16  redshift_measured  16 non-

## 1. Parameters

### 1.1 Brightness Temperature
$$
    T_B = \frac{S_\nu D_L^2}{2\pi k_B (\nu\Delta t)^2(1+z)}
$${#eq-brightness-temp}
where:
- $S_\nu$ = peak specific flux, in Jy
- $D_L$ = luminosity distance, in Gpc
- $\Delta t$ = burst duration, in ms
- $\nu$ = central frequency, in GHz
- $k_B$ = Boltzmann constant
- $z$ = redshift

In [47]:
H_0 = 67.4*1000*100  # cm s^-1 Mpc^-1
c = 29979245800 # cm s^-1
Mpc_to_cm = 3.085677581491367E24
Gyr_to_s = 3.15576E16
Hubble_time = 1/(H_0 / Mpc_to_cm * Gyr_to_s)  # Gyr
Hubble_distance = c/H_0  # Mpc
Omega_b = 0.0224/((H_0)/1000/100/100)**2
Omega_m = 0.315
Omega_Lambda = 0.685
f_IGM = 0.83
chi = 7/8
G = 6.6743e-8 # cm^3 g^-1 s^-2
m_p = 1.67262192e-24 # g
dm_factor = 3*c*H_0/(Mpc_to_cm)**2*1e6*Omega_b*f_IGM*chi/(8*np.pi*G*m_p)
DM_host_lab = 70.0 # pc cm^-3
DM_halo = 30.0

def comoving_distance_at_z(z): # Mpc
    zp1 = 1.0+z
    h0_up = np.sqrt(1+Omega_m/Omega_Lambda) * scipy.special.hyp2f1(1/3,1/2,4/3,-Omega_m/Omega_Lambda)
    hz_up = zp1 * np.sqrt(1+Omega_m*zp1**3/Omega_Lambda) * scipy.special.hyp2f1(1/3,1/2,4/3,-Omega_m*zp1**3/Omega_Lambda)
    h0_down = np.sqrt(Omega_Lambda + Omega_m)
    hz_down = np.sqrt(Omega_Lambda + Omega_m * zp1**3)
    return Hubble_distance * (hz_up/hz_down-h0_up/h0_down)


def luminosity_distance_at_z(z): # Mpc
    return (1. + z) * comoving_distance_at_z(z)

# ? I am not sure why it is log_10 of the value when there is none in formula?
#   Maybe need to do some pen and paper.
catalog['brightness_temperature'] = np.log10(1.1e35 
                            * catalog['flux'] 
                            * (catalog['width']*1000)**(-2) 
                            * (catalog['frequency']/1000)**(-2) 
                            * (luminosity_distance_at_z(catalog['redshift'])/1000)**2 / (1+catalog['redshift'])
                        )

## 1.2 Redshift Corrected Frequency
A redshift is calculated using:
$$
    1 + z = \frac{f_\text{emit}}{f_\text{obs}}
$$
therefore, the emission frequency is:
$$
    f_\text{emit} = f_\text{obs}(1 + z)
$$
where we take the observed frequency to be the peak frequency.

In [53]:
catalog['rest_frequency'] = catalog['frequency'] * (1 + catalog['redshift'])

## 1.3 Energy
$$
    E = 4\pi \frac{D_L^2 F \nu_c}{1-z}
$${#eq-burst-energy}
where:
- $E$ = energy, in erg
- $F$ = fluence, in Jy ms
- $\nu_c$ = peak frequence, in MHz

In [77]:
# TODO Check units of all these calculations
catalog['energy'] = 1e-23 * catalog['frequency'] * 1e6 * catalog['fluence'] / 1000 * (4*np.pi*(luminosity_distance_at_z(catalog['redshift'])*Mpc_to_cm)**2) / (1+catalog['redshift'])